In [ ]:
import torch
import cv2
import numpy as np
from facenet_pytorch import MTCNN, InceptionResnetV1

# Initialize MTCNN (Face Detection)
mtcnn = MTCNN()

# Initialize Inception Resnet (Face Recognition)
model = InceptionResnetV1(pretrained='vggface2').eval()

# Load the webcam
cap = cv2.VideoCapture(0)

# Function to save the face embedding
def save_face_embedding(face_tensor):
    # Get the face embedding
    embedding = model(face_tensor)
    
    # Save the embedding to a file (in .pth format)
    torch.save(embedding, 'my_face_embeddings.pth')
    print("Face embedding saved successfully!")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Detect faces
    faces, _ = mtcnn.detect(frame)
    
    if faces is not None:
        for face in faces:
            # Draw bounding box around the detected face
            cv2.rectangle(frame, 
                          (int(face[0]), int(face[1])), 
                          (int(face[2]), int(face[3])), 
                          (0, 255, 0), 2)
            
            # Crop the face from the frame
            face_crop = frame[int(face[1]):int(face[3]), int(face[0]):int(face[2])]
            
            if face_crop.size != 0:
                # Resize the face crop to 160x160 pixels (required for FaceNet)
                face_crop_resized = cv2.resize(face_crop, (160, 160))

                # Convert to RGB
                face_crop_rgb = cv2.cvtColor(face_crop_resized, cv2.COLOR_BGR2RGB)

                # Convert to PyTorch tensor
                face_tensor = torch.FloatTensor(face_crop_rgb).permute(2, 0, 1)

                # Normalize the image to [-1, 1]
                face_tensor = (face_tensor / 255.0) * 2 - 1

                # Add batch dimension
                face_tensor = face_tensor.unsqueeze(0)

                # Ask for confirmation to save the embedding
                cv2.putText(frame, 'Press "s" to save your face', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

        # Show the frame
        cv2.imshow('Face Recognition - Save Face', frame)

    # Check if 's' key is pressed to save the embedding
    if cv2.waitKey(1) & 0xFF == ord('s'):
        save_face_embedding(face_tensor)

    # Exit when the 'Esc' key is pressed (key code 27)
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import torch
import cv2
import numpy as np
from facenet_pytorch import MTCNN, InceptionResnetV1
from scipy.spatial.distance import cosine

# Initialize MTCNN (Face Detection)
mtcnn = MTCNN()

# Initialize Inception Resnet (Face Recognition)
model = InceptionResnetV1(pretrained='vggface2').eval()

# Load the saved face embedding (if it exists)
saved_embedding = torch.load('my_face_embeddings.pth')

# Load the webcam
cap = cv2.VideoCapture(0)

# Threshold for face recognition (cosine distance threshold)
threshold = 0.6


# Function to recognize face by comparing embeddings
def recognize_face(face_tensor):
    # Get the face embedding
    embedding = model(face_tensor)
    
    # Flatten both the saved embedding and the current face embedding to 1D
    embedding_flat = embedding.flatten()
    saved_embedding_flat = saved_embedding.flatten()
    
    # Calculate the cosine distance between saved embedding and the current face embedding
    distance = cosine(saved_embedding_flat.detach().numpy(), embedding_flat.detach().numpy())
    print(f'Cosine distance: {distance}')
    
    if distance < threshold:
        print("User recognized: Gaindu")  # Print the recognized user on the console
        return "Gaindu"
    else:
        print("User recognized: Unknown")  # Print "Unknown" on the console if not recognized
        return "Unknown"

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Detect faces
    faces, _ = mtcnn.detect(frame)
    
    if faces is not None:
        for face in faces:
            # Draw bounding box around the detected face
            cv2.rectangle(frame, 
                          (int(face[0]), int(face[1])), 
                          (int(face[2]), int(face[3])), 
                          (0, 255, 0), 2)
            
            # Crop the face from the frame
            face_crop = frame[int(face[1]):int(face[3]), int(face[0]):int(face[2])]
            
            if face_crop.size != 0:
                # Resize the face crop to 160x160 pixels (required for FaceNet)
                face_crop_resized = cv2.resize(face_crop, (160, 160))

                # Convert to RGB
                face_crop_rgb = cv2.cvtColor(face_crop_resized, cv2.COLOR_BGR2RGB)

                # Convert to PyTorch tensor
                face_tensor = torch.FloatTensor(face_crop_rgb).permute(2, 0, 1)

                # Normalize the image to [-1, 1]
                face_tensor = (face_tensor / 255.0) * 2 - 1

                # Add batch dimension
                face_tensor = face_tensor.unsqueeze(0)

                # Recognize the face
                label = recognize_face(face_tensor)

                # Display the label on the image
                cv2.putText(frame, label, (int(face[0]), int(face[1]) - 10), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

        # Show the frame with bounding boxes and labels
        cv2.imshow('Face Recognition', frame)

    # Exit when the 'Esc' key is pressed (key code 27)
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()
